## Dataset #1: Web Scraping
---
Web scraping of https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming 

The goal is to gather the Netflix shows presented on this wiki page.

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def download_dataset1(out_file1: str, sample_p: bool):
    url = 'https://en.wikipedia.org/wiki/List_of_ended_Netflix_original_programming'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.find_all('table', class_='wikitable')
    data = []

    # Loop through each table and extract the desired columns
    for table in tables[:-1]:
        rows = table.find_all('tr')[1:]  # exclude the header row
        headers = table.find_all('th')
        title_index, genre_index, date_index = None, None, None
        for i, header in enumerate(headers):
            if header.text.strip() == 'Title':
                title_index = i
            elif header.text.strip() == 'Genre':
                genre_index = i
            elif header.text.strip() in ['Premiere', 'Release date']:
                date_index = i

        for row in rows:
            columns = row.find_all('td')
            title = columns[title_index].text.strip() if title_index is not None else ''
            genre = columns[genre_index].text.strip() if (genre_index is not None and columns[genre_index]) else 'Family film'  #add family film if no genre (on wiki page, only this table is empty)
            date = columns[date_index].text.strip() if date_index is not None else ''
            data.append([title, genre, date])

    netflix_movie = pd.DataFrame(data, columns=['Title', 'Genre', 'Premiere Date'])
    if sample_p == True:
        netflix_movie = netflix_movie.head(5)
        netflix_movie.to_csv(out_file1, index=False)
    
    elif sample_p == False:
        netflix_movie.to_csv(out_file1, index=False)
    return netflix_movie

In [61]:
#testing to get the frst 5 entries (True) ; if want the whole dataset you put False
download_dataset1('dataset1.csv', True)

,Title,Genre,Premiere Date
0,House of Cards,Political drama,"February 1, 2013"
1,Hemlock Grove,Horror/thriller,"April 19, 2013"
2,Orange Is the New Black,Comedy drama,"July 11, 2013"
3,Marco Polo,Historical drama,"December 12, 2014"
4,Bloodline,Thriller,"March 20, 2015"


## Dataset #2: OMDB API
---
OMDB API: https://www.omdbapi.com/apikey.aspx

This API has a limit of 1,000 request daily for 1 user. My project whole dataset consists of 1000+ data so I had to request for 2 different API Keys (using two different emails). If you know your rows are <1000, have key1 and key2 as the same values 



In [62]:
import pandas as pd
import csv
import requests

def get_API(in_file, key):
    OMDB_API_KEY = key      #this is the API key that is unique to user (need to request for it)
    OMDB_API_URL = "https://www.omdbapi.com/"

    # Define the parameters to request for each movie
    params = {
        "apikey": OMDB_API_KEY,
        "plot": "short",
        "r": "json",
    }
    
    updated_rows = []
    
    for i, row in enumerate(in_file):
        if i == 0:
            #header row of the CSV file, add new column headers for the movie information
            row.extend(["Language", "Country", "IMDb Rating", "IMDb Votes"])
            updated_rows.append(row)
        else:
            # get movie information data from the OMDb API
            title = row[0]
            params["t"] = title
            response = requests.get(OMDB_API_URL, params=params)
            movie_data = response.json()

            language = movie_data.get("Language")
            country = movie_data.get("Country")
            imdb_rating = movie_data.get("imdbRating")
            imdb_votes = movie_data.get("imdbVotes")

            # Add new columns to the current row and append the updated row to the list
            row.extend([language, country, imdb_rating, imdb_votes])
            updated_rows.append(row)

    return updated_rows

def download_dataset2(in_file1: str, out_file2: str, sample_p: bool, key1:str, key2:str):
    # Initialize updated rows for both halves
    updated_rows1 = []
    updated_rows2 = []

    with open(out_file2, "w", newline="", encoding="utf-8") as outfile:
        output = csv.writer(outfile)

        if sample_p == True:
            with open(in_file1, newline="", encoding="utf-8") as csvfile:
                csvreader = csv.reader(csvfile)
                first_six_rows = [next(csvreader)]
                for i in range(5):
                    first_six_rows.append(next(csvreader))               #first 6 row including the header - 5 movie records
                updated_rows = get_API(first_six_rows, key1)
                output.writerows(updated_rows)

        #would be useful if original dataset has 1000-2000 records because 2 API KEYs max
        elif sample_p == False:
            # Splitting the input CSV file into two halves to avoid 1,000+ records
            df = pd.read_csv(in_file1)
            half_size = len(df) // 2
            size_first_half = half_size + len(df) % 2
            size_second_half = half_size

            #input file object to the get_API() for each half
            with open(in_file1, newline='', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                first_half = list(reader)[:size_first_half]
                updated_rows1 = get_API(first_half,key1)

            with open(in_file1, newline='', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                second_half = list(reader)[size_first_half:size_first_half+size_second_half+1]
                updated_rows2 = get_API(second_half,key2)

            updated_rows = updated_rows1 + updated_rows2
            output.writerows(updated_rows)

        # Convert the updated data to a DataFrame
        updated_df = pd.DataFrame(updated_rows, columns=["Title","Genre","Premiere Date", "Language", "Country", "IMDb Rating", "IMDb Votes"])
        updated_df = updated_df.drop(index=0)                       #the first row is a duplicate of the title?

        return updated_df


In [63]:
#testing dataset2
#'24bea5c3','b17bad04' are my unique keys from OMDB API website
download_dataset2('dataset1.csv','dataset2.csv',True,'b17bad04','24bea5c3')


,Title,Genre,Premiere Date,Language,Country,IMDb Rating,IMDb Votes
1,House of Cards,Political drama,"February 1, 2013",English,United States,8.7,"511,346"
2,Hemlock Grove,Horror/thriller,"April 19, 2013",English,United States,7.0,"41,130"
3,Orange Is the New Black,Comedy drama,"July 11, 2013",English,United States,8.1,"308,035"
4,Marco Polo,Historical drama,"December 12, 2014","English, Cantonese, Mandarin, Mongolian, Persi...",United States,8.0,"74,805"
5,Bloodline,Thriller,"March 20, 2015",English,United States,7.9,"54,639"



## Dataset 3: Yahoo Finance API

---
getting the stock prices of Netflix and merge it together with the respective rows of the movies/films in the dataset. But first, we need to figure out what's the max and min dates of the dataset 

In [65]:
#checking what's the max and min date of the whole dataset (this part is just to see what the the overall range)
import csv
from datetime import datetime


def find_minMax(in_file):
    max_date = None
    min_date = None

    with open(in_file, 'r') as file:
        data = csv.reader(file)
        next(data)                                            # skip header row
        for row in data:
            date_str = row[2]                                   # assuming the date is in the third column
            date = datetime.strptime(date_str, '%B %d, %Y')     # parse the date string
            if max_date is None or date > max_date:
                max_date = date
            if min_date is None or date < min_date:
                min_date = date

    print("Maximum date:", max_date)
    print("Minimum date:", min_date)

find_minMax('dataset2.csv')     #should use dataset 2 just to keep it consistent  (Dataset2 comes from download_dataset2 function above)

Maximum date: 2023-04-14 00:00:00
Minimum date: 2012-02-06 00:00:00


Using Yahoo Finance API

In [43]:
import yfinance as yf
import pandas as pd

def download_dataset3(in_file2: str, out_file3:str, sample_p: bool):
    ticker = "NFLX"                                 #symbol for netflix
    start_date = "2012-02-06"
    end_date = "2023-04-14"

    stock_data = yf.download(ticker, start=start_date, end=end_date)
    close_prices = stock_data["Close"]
    stock_df = pd.DataFrame({"Date": close_prices.index, "Close Price": close_prices.values})

    movie_df = pd.read_csv(in_file2)

    movie_df['Premiere Date'] = pd.to_datetime(movie_df['Premiere Date'], format='%B %d, %Y')
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])

   # Merge both dataframes based on date (premiere date on csv and "date" from stock df)
    merged_df = pd.merge(movie_df, stock_df, how='left', left_on='Premiere Date', right_on='Date')
    merged_df = merged_df.drop(columns=['Date'])  # Drop the redundant date column

    if sample_p == True:
        merged_df.head(5).to_csv(out_file3, index=False)    #only first 5 rows to CSV
        merged_df = merged_df.head(5)
    else:
        merged_df.to_csv(out_file3, index=False)            #all rows to CSV

    show_df = merged_df[["Title","Genre","Premiere Date", "Language", "Country", "IMDb Rating", "IMDb Votes", "Close Price"]]
    return show_df


In [59]:
#testing dataset3
download_dataset3('dataset2.csv', 'dataset3.csv', True)

[*********************100%***********************]  1 of 1 completed


,Title,Genre,Premiere Date,Language,Country,IMDb Rating,IMDb Votes,Close Price
0,House of Cards,Political drama,2013-02-01,English,United States,8.7,"511,346",23.542856
1,Hemlock Grove,Horror/thriller,2013-04-19,English,United States,7.0,"41,130",23.338572
2,Orange Is the New Black,Comedy drama,2013-07-11,English,United States,8.1,"308,035",34.881428
3,Marco Polo,Historical drama,2014-12-12,"English, Cantonese, Mandarin, Mongolian, Persi...",United States,8.0,"74,805",47.782856
4,Bloodline,Thriller,2015-03-20,English,United States,7.9,"54,639",61.185715
